In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.nddata import CCDData

In [ ]:
import numpy as np


In [ ]:
import sys
sys.path.append("../imaging")
sys.path.append("../photometry")
from phot_utils import xmatch

In [ ]:
import tomllib

In [ ]:
import arya

In [ ]:
import read_iso

In [ ]:
import seaborn as sns

In [ ]:
import shapely

In [ ]:
obj = "yasone1"

In [ ]:
if obj == "yasone1":
    ra0, dec0 = 265.52019, 13.17146
elif obj == "yasone2":
    ra0, dec0 = 262.34921, 6.42302
elif obj == "yasone3":
    ra0, dec0 = 292.96258, -26.44994

In [ ]:
def get_mag_shift(objname):
    shifts = {}
    with open("../photometry/" + objname + "/panstarrs_shift.toml", "rb") as f:
        shifts = tomllib.load(f)
    return shifts

In [ ]:
def read_catalogue(objname):
    cat = Table.read(f"../photometry/{objname}/allcolours.cat")

    shifts = get_mag_shift(objname)
    cat["GMAG"] -= shifts["g"]
    cat["RMAG"] -= shifts["r"]
    cat["IMAG"] -= shifts["i"]
    cat = cat[(cat["RFLAGS"] < 8 ) & (cat["RFLAGS_WEIGHT"] == 0) ]
    return cat

In [ ]:
def read_catalogue_julen(objname):
    cat = Table.read(f"{objname}_julen.cat")

    return cat

In [ ]:
get_mag_shift(obj)

In [ ]:
cat = read_catalogue(obj)

In [ ]:
cat_julen = read_catalogue_julen(obj)

In [ ]:
xmatch_idx, xmatch_dist, xmatch_filt = xmatch(cat["ALPHA_J2000"], cat["DELTA_J2000"], cat_julen["ALPHA_J2000"], cat_julen["DELTA_J2000"])

In [ ]:
plt.scatter(cat["ALPHA_J2000"], cat["DELTA_J2000"], s=2)
plt.scatter(cat_julen["ALPHA_J2000"], cat_julen["DELTA_J2000"], s=1)


Abovse shows a plot of detected sources through both methods.

In [ ]:
plt.scatter(cat["ALPHA_J2000"][xmatch_filt], 3600*(cat_julen["ALPHA_J2000"][xmatch_idx[xmatch_filt]] - cat["ALPHA_J2000"][xmatch_filt]), label="delta ra")
plt.scatter(cat["ALPHA_J2000"][xmatch_filt], 3600*(cat_julen["DELTA_J2000"][xmatch_idx[xmatch_filt]] - cat["DELTA_J2000"][xmatch_filt]), label="delta dec")
plt.xlabel("RA")
plt.ylabel("astrometric difference / arcsec")
arya.Legend(-1)

In [ ]:
plt.scatter(cat["GMAG"][xmatch_filt], cat_julen["GMAG"][xmatch_idx[xmatch_filt]] - cat["GMAG"][xmatch_filt], s=1, color=arya.COLORS[2], label="g")


plt.scatter(cat["RMAG"][xmatch_filt], cat_julen["RMAG"][xmatch_idx[xmatch_filt]] - cat["RMAG"][xmatch_filt], s=1, color=arya.COLORS[3], label="r")


plt.scatter(cat["IMAG"][xmatch_filt], cat_julen["IMAG"][xmatch_idx[xmatch_filt]] - cat["IMAG"][xmatch_filt], s=1, color=arya.COLORS[1], label="i")

plt.axhline(0, color="black")

plt.xlabel("magnitude")
plt.ylabel("residual (julen - me)")
plt.ylim(-1, 0.5)
arya.Legend(-1)

In [ ]:
plt.scatter(cat["FWHM_IMAGE"][xmatch_filt], cat_julen["fwhm"][xmatch_idx[xmatch_filt]], s=1, alpha=0.3)
# plt.xscale("log")
# plt.yscale("log")

plt.plot([0, 10], [0, 10], color="k")
plt.xlim(0, 10)
plt.ylim(0, 10)
plt.xlabel("FWHM me / pixels")
plt.ylabel("FWHM julen / pixels")

From above, the FWHM determinations are comperable but not strongly correlated. As the median FWHM is around 4 pixels, and Julen uses an aperture 1.7 times the FWHM, a fixed aperture would be around 7 pixels.

In [ ]:
plt.hist(cat["RMAG"], np.linspace(15, 28, 50))

In [ ]:
plt.scatter(cat["RMAG"], cat["RMAG_ERR"], s=1, color=arya.COLORS[0], label="15x poisson (me)")
plt.ylim(0, 1)
plt.xlim(15, 28)
plt.axhline(0.2, color="k")

In [ ]:
# plt.scatter(cat["RMAG"], cat["RMAG_ERR"] , s=1, color=arya.COLORS[0], label="me")
bkg_err = 500
# plt.scatter(cat["RMAG"], 0.02 + bkg_err / cat["FLUX_APER"][:, 1] * 2.5 / np.log(10) , s=1, color=arya.COLORS[0], label="me")
plt.scatter(cat["RMAG"], 0.01 + 15 / cat["FLUX_APER"][:, 4]**0.5 * 2.5 / np.log(10) , s=1, color=arya.COLORS[0], label="15x poisson (me)")
plt.scatter(cat_julen["RMAG"], cat_julen["RMAG_ERR"], s=1, color=arya.COLORS[1], label="julen")
plt.scatter(cat_julen["RMAG"], 0.01 / np.sqrt(10**(0.4*(-cat_julen["RMAG"]+18))), s=0.1, color=arya.COLORS[2], label="rescaled poisson error (julen)")
plt.legend()
plt.ylim(0, 1)
plt.xlim(15, 28)
plt.xlabel("r mag")
plt.ylabel("r mag err")
plt.axhline(0.2)

Julen's data process calculates uncertainties through sep. Unfortunantly, since the sausero package rescales
the fluxes by the zeropoint, so the poisson uncertainty is substantially amplified and dominates the uncertainty budget. The below plot shows that the uncertainty is just related to the flux value---Julen's uncertainties are not reliable :/

In [ ]:
zeropoint = np.nanmedian((cat["RMAG"] - cat["MAG_APER"][:, 3])[~cat["RMAG"].mask])

In [ ]:
zeropoint

In [ ]:
plt.scatter(cat["RMAG"], cat["MAG_AUTO"] - cat["RMAG"] + zeropoint, s=1, alpha=0.05)
plt.axhline(0, color="k")
plt.ylim(-1, 1)
plt.xlim(18, 26)

In [ ]:
plt.scatter(cat["RMAG"], cat["MAG_ISO"] - cat["RMAG"] + zeropoint, s=1, alpha=0.05)
plt.axhline(0, color="k")
plt.ylim(-1, 1)
plt.xlim(18, 26)

In [ ]:
plt.scatter(cat["RMAG"], cat["MAG_WIN"] - cat["RMAG"] + zeropoint, s=1, alpha=0.05)
plt.axhline(0, color="k")
plt.ylim(-1, 1)
plt.xlim(18, 26)